# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [3]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [6]:
 len(data[data['TARGET'] == 1]) /  len(data['TARGET']) 

0.1190304145043684

Обучим логистическую регрессию с параметрами по умолчанию.

In [4]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [8]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [11]:
from sklearn.metrics import f1_score
f1_score(ytest, prediction)

0.0

**Вопрос**  
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [12]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.86331972, 0.13668028],
       [0.86305957, 0.13694043],
       [0.80967732, 0.19032268],
       [0.73110167, 0.26889833],
       [0.87780414, 0.12219586],
       [0.96122864, 0.03877136],
       [0.95965491, 0.04034509],
       [0.96560659, 0.03439341],
       [0.92114871, 0.07885129],
       [0.9349701 , 0.0650299 ]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [19]:
proba = probs_test[:,1]
predict = (proba >= 0.1 ).astype(int)
f1_score(ytest, predict)

0.2469521725539231

У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [35]:
coef = lr.coef_.reshape(-1)

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [39]:
pd.DataFrame({'feature' : X.columns, 'Важность' : coef }).sort_values('Важность')[::-1]

,feature,Важность
6,PERSONAL_INCOME,0.000011
9,LOAN_DLQ_NUM,-0.000015
2,SOCSTATUS_PENS_FL,-0.000428
5,DEPENDANTS,-0.000496
1,SOCSTATUS_WORK_FL,-0.000791
3,GENDER,-0.001072
8,LOAN_NUM_CLOSED,-0.001268
4,CHILD_TOTAL,-0.001350
7,LOAN_NUM_TOTAL,-0.001800
0,AGE,-0.055521


## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
p = np.linspace(0,1, 1000)
maxim = -10000000




**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [1]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)
f1_s = []
for c in range(1, 10):
    for pe in p:
        model = LogisticRegression(C=c)
        model.fit(Xtrain_new, ytrain_new)
        predict = model.predict_proba(Xval)[:,1]
        predict = (predict >= pe).astype(int)
        f1 = f1_score(yval, predict)
        f1_s.append(f1)
plt.plot(f1_s)

NameError: name 'train_test_split' is not defined

In [ ]:
model = LogisticRegression(C = ...)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)

classes = ... # переведите полученные вероятности в классы по найденному порогу

In [ ]:
# вычислите значение f1_score на тестовых данных